In [1]:
# Copyright (c) 2018-present, Facebook, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
##############################################################################
"""YACS -- Yet Another Configuration System is designed to be a simple
configuration management system for academic and industrial research
projects.

See README.md for usage and examples.
"""

import copy
import io
import logging
import os
from ast import literal_eval

import yaml
import importlib.util

# Filename extensions for loading configs from files
_YAML_EXTS = {"", ".yaml", ".yml"}
_PY_EXTS = {".py"}

_FILE_TYPES = (io.IOBase,)

# CfgNodes can only contain a limited set of valid types
_VALID_TYPES = {tuple, list, str, int, float, bool, type(None)}


logger = logging.getLogger(__name__)


class CfgNode(dict):
    """
    CfgNode represents an internal node in the configuration tree. It's a simple
    dict-like container that allows for attribute-based access to keys.
    """

    IMMUTABLE = "__immutable__"
    DEPRECATED_KEYS = "__deprecated_keys__"
    RENAMED_KEYS = "__renamed_keys__"
    NEW_ALLOWED = "__new_allowed__"

    def __init__(self, init_dict=None, key_list=None, new_allowed=False):
        """
        Args:
            init_dict (dict): the possibly-nested dictionary to initailize the
                CfgNode.
            key_list (list[str]): a list of names which index this CfgNode from
                the root.
                Currently only used for logging purposes.
            new_allowed (bool): whether adding new key is allowed when merging with
                other configs.
        """
        # Recursively convert nested dictionaries in init_dict into CfgNodes
        init_dict = {} if init_dict is None else init_dict
        key_list = [] if key_list is None else key_list
        init_dict = self._create_config_tree_from_dict(init_dict, key_list)
        super(CfgNode, self).__init__(init_dict)
        # Manage if the CfgNode is frozen or not
        self.__dict__[CfgNode.IMMUTABLE] = False
        # Deprecated options
        # If an option is removed from the code and you don't want to break existing
        # yaml configs, you can add the full config key as a string to the set below.
        self.__dict__[CfgNode.DEPRECATED_KEYS] = set()
        # Renamed options
        # If you rename a config option, record the mapping from the old name to the
        # new name in the dictionary below. Optionally, if the type also changed, you
        # can make the value a tuple that specifies first the renamed key and then
        # instructions for how to edit the config file.
        self.__dict__[CfgNode.RENAMED_KEYS] = {
            # 'EXAMPLE.OLD.KEY': 'EXAMPLE.NEW.KEY',  # Dummy example to follow
            # 'EXAMPLE.OLD.KEY': (                   # A more complex example to follow
            #     'EXAMPLE.NEW.KEY',
            #     "Also convert to a tuple, e.g., 'foo' -> ('foo',) or "
            #     + "'foo:bar' -> ('foo', 'bar')"
            # ),
        }

        # Allow new attributes after initialisation
        self.__dict__[CfgNode.NEW_ALLOWED] = new_allowed

    @classmethod
    def _create_config_tree_from_dict(cls, dic, key_list):
        """
        Create a configuration tree using the given dict.
        Any dict-like objects inside dict will be treated as a new CfgNode.

        Args:
            dic (dict):
            key_list (list[str]): a list of names which index this CfgNode from
                the root. Currently only used for logging purposes.
        """
        dic = copy.deepcopy(dic)
        for k, v in dic.items():
            if isinstance(v, dict):
                # Convert dict to CfgNode
                dic[k] = cls(v, key_list=key_list + [k])
            else:
                # Check for valid leaf type or nested CfgNode
                _assert_with_logging(
                    _valid_type(v, allow_cfg_node=False),
                    "Key {} with value {} is not a valid type; valid types: {}".format(
                        ".".join(key_list + [k]), type(v), _VALID_TYPES
                    ),
                )
        return dic

    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        if self.is_frozen():
            raise AttributeError(
                "Attempted to set {} to {}, but CfgNode is immutable".format(
                    name, value
                )
            )

        _assert_with_logging(
            name not in self.__dict__,
            "Invalid attempt to modify internal CfgNode state: {}".format(name),
        )
        _assert_with_logging(
            _valid_type(value, allow_cfg_node=True),
            "Invalid type {} for key {}; valid types = {}".format(
                type(value), name, _VALID_TYPES
            ),
        )

        self[name] = value

    def __str__(self):
        def _indent(s_, num_spaces):
            s = s_.split("\n")
            if len(s) == 1:
                return s_
            first = s.pop(0)
            s = [(num_spaces * " ") + line for line in s]
            s = "\n".join(s)
            s = first + "\n" + s
            return s

        r = ""
        s = []
        for k, v in sorted(self.items()):
            seperator = "\n" if isinstance(v, CfgNode) else " "
            attr_str = "{}:{}{}".format(str(k), seperator, str(v))
            attr_str = _indent(attr_str, 2)
            s.append(attr_str)
        r += "\n".join(s)
        return r

    def __repr__(self):
        return "{}({})".format(self.__class__.__name__, super(CfgNode, self).__repr__())

    def convert_to_dict(self, cfg_node, key_list):
        if not isinstance(cfg_node, CfgNode):
            _assert_with_logging(
                _valid_type(cfg_node),
                "Key {} with value {} is not a valid type; valid types: {}".format(
                    ".".join(key_list), type(cfg_node), _VALID_TYPES
                ),
            )
            return cfg_node
        else:
            cfg_dict = dict(cfg_node)
            for k, v in cfg_dict.items():
                cfg_dict[k] = self.convert_to_dict(v, key_list + [k])
            return cfg_dict

    def dump(self, **kwargs):
        """Dump to a string."""

        def convert_to_dict(cfg_node, key_list):
            if not isinstance(cfg_node, CfgNode):
                _assert_with_logging(
                    _valid_type(cfg_node),
                    "Key {} with value {} is not a valid type; valid types: {}".format(
                        ".".join(key_list), type(cfg_node), _VALID_TYPES
                    ),
                )
                return cfg_node
            else:
                cfg_dict = dict(cfg_node)
                for k, v in cfg_dict.items():
                    cfg_dict[k] = convert_to_dict(v, key_list + [k])
                return cfg_dict

        self_as_dict = convert_to_dict(self, [])
        return yaml.safe_dump(self_as_dict, **kwargs)

    def merge_from_file(self, cfg_filename):
        """Load a yaml config file and merge it this CfgNode."""
        with open(cfg_filename, "r", encoding="utf-8") as f:
            cfg = self.load_cfg(f)
        self.merge_from_other_cfg(cfg)

    def merge_from_other_cfg(self, cfg_other):
        """Merge `cfg_other` into this CfgNode."""
        _merge_a_into_b(cfg_other, self, self, [])

    def merge_from_list(self, cfg_list):
        """Merge config (keys, values) in a list (e.g., from command line) into
        this CfgNode. For example, `cfg_list = ['FOO.BAR', 0.5]`.
        """
        _assert_with_logging(
            len(cfg_list) % 2 == 0,
            "Override list has odd length: {}; it must be a list of pairs".format(
                cfg_list
            ),
        )
        root = self
        for full_key, v in zip(cfg_list[0::2], cfg_list[1::2]):
            if root.key_is_deprecated(full_key):
                continue
            if root.key_is_renamed(full_key):
                root.raise_key_rename_error(full_key)
            key_list = full_key.split(".")
            d = self
            for subkey in key_list[:-1]:
                _assert_with_logging(
                    subkey in d, "Non-existent key: {}".format(full_key)
                )
                d = d[subkey]
            subkey = key_list[-1]
            _assert_with_logging(subkey in d, "Non-existent key: {}".format(full_key))
            value = self._decode_cfg_value(v)
            value = _check_and_coerce_cfg_value_type(value, d[subkey], subkey, full_key)
            d[subkey] = value

    def freeze(self):
        """Make this CfgNode and all of its children immutable."""
        self._immutable(True)

    def defrost(self):
        """Make this CfgNode and all of its children mutable."""
        self._immutable(False)

    def is_frozen(self):
        """Return mutability."""
        return self.__dict__[CfgNode.IMMUTABLE]

    def _immutable(self, is_immutable):
        """Set immutability to is_immutable and recursively apply the setting
        to all nested CfgNodes.
        """
        self.__dict__[CfgNode.IMMUTABLE] = is_immutable
        # Recursively set immutable state
        for v in self.__dict__.values():
            if isinstance(v, CfgNode):
                v._immutable(is_immutable)
        for v in self.values():
            if isinstance(v, CfgNode):
                v._immutable(is_immutable)

    def clone(self):
        """Recursively copy this CfgNode."""
        return copy.deepcopy(self)

    def register_deprecated_key(self, key):
        """Register key (e.g. `FOO.BAR`) a deprecated option. When merging deprecated
        keys a warning is generated and the key is ignored.
        """
        _assert_with_logging(
            key not in self.__dict__[CfgNode.DEPRECATED_KEYS],
            "key {} is already registered as a deprecated key".format(key),
        )
        self.__dict__[CfgNode.DEPRECATED_KEYS].add(key)

    def register_renamed_key(self, old_name, new_name, message=None):
        """Register a key as having been renamed from `old_name` to `new_name`.
        When merging a renamed key, an exception is thrown alerting to user to
        the fact that the key has been renamed.
        """
        _assert_with_logging(
            old_name not in self.__dict__[CfgNode.RENAMED_KEYS],
            "key {} is already registered as a renamed cfg key".format(old_name),
        )
        value = new_name
        if message:
            value = (new_name, message)
        self.__dict__[CfgNode.RENAMED_KEYS][old_name] = value

    def key_is_deprecated(self, full_key):
        """Test if a key is deprecated."""
        if full_key in self.__dict__[CfgNode.DEPRECATED_KEYS]:
            logger.warning("Deprecated config key (ignoring): {}".format(full_key))
            return True
        return False

    def key_is_renamed(self, full_key):
        """Test if a key is renamed."""
        return full_key in self.__dict__[CfgNode.RENAMED_KEYS]

    def raise_key_rename_error(self, full_key):
        new_key = self.__dict__[CfgNode.RENAMED_KEYS][full_key]
        if isinstance(new_key, tuple):
            msg = " Note: " + new_key[1]
            new_key = new_key[0]
        else:
            msg = ""
        raise KeyError(
            "Key {} was renamed to {}; please update your config.{}".format(
                full_key, new_key, msg
            )
        )

    def is_new_allowed(self):
        return self.__dict__[CfgNode.NEW_ALLOWED]

    @classmethod
    def load_cfg(cls, cfg_file_obj_or_str):
        """
        Load a cfg.
        Args:
            cfg_file_obj_or_str (str or file):
                Supports loading from:
                - A file object backed by a YAML file
                - A file object backed by a Python source file that exports an attribute
                  "cfg" that is either a dict or a CfgNode
                - A string that can be parsed as valid YAML
        """
        _assert_with_logging(
            isinstance(cfg_file_obj_or_str, _FILE_TYPES + (str,)),
            "Expected first argument to be of type {} or {}, but it was {}".format(
                _FILE_TYPES, str, type(cfg_file_obj_or_str)
            ),
        )
        if isinstance(cfg_file_obj_or_str, str):
            return cls._load_cfg_from_yaml_str(cfg_file_obj_or_str)
        elif isinstance(cfg_file_obj_or_str, _FILE_TYPES):
            return cls._load_cfg_from_file(cfg_file_obj_or_str)
        else:
            raise NotImplementedError("Impossible to reach here (unless there's a bug)")

    @classmethod
    def _load_cfg_from_file(cls, file_obj):
        """Load a config from a YAML file or a Python source file."""
        _, file_extension = os.path.splitext(file_obj.name)
        if file_extension in _YAML_EXTS:
            return cls._load_cfg_from_yaml_str(file_obj.read())
        elif file_extension in _PY_EXTS:
            return cls._load_cfg_py_source(file_obj.name)
        else:
            raise Exception(
                "Attempt to load from an unsupported file type {}; "
                "only {} are supported".format(file_obj, _YAML_EXTS.union(_PY_EXTS))
            )

    @classmethod
    def _load_cfg_from_yaml_str(cls, str_obj):
        """Load a config from a YAML string encoding."""
        cfg_as_dict = yaml.safe_load(str_obj)
        return cls(cfg_as_dict)

    @classmethod
    def _load_cfg_py_source(cls, filename):
        """Load a config from a Python source file."""
        module = _load_module_from_file("yacs.config.override", filename)
        _assert_with_logging(
            hasattr(module, "cfg"),
            "Python module from file {} must have 'cfg' attr".format(filename),
        )
        VALID_ATTR_TYPES = {dict, CfgNode}
        _assert_with_logging(
            type(module.cfg) in VALID_ATTR_TYPES,
            "Imported module 'cfg' attr must be in {} but is {} instead".format(
                VALID_ATTR_TYPES, type(module.cfg)
            ),
        )
        return cls(module.cfg)

    @classmethod
    def _decode_cfg_value(cls, value):
        """
        Decodes a raw config value (e.g., from a yaml config files or command
        line argument) into a Python object.

        If the value is a dict, it will be interpreted as a new CfgNode.
        If the value is a str, it will be evaluated as literals.
        Otherwise it is returned as-is.
        """
        # Configs parsed from raw yaml will contain dictionary keys that need to be
        # converted to CfgNode objects
        if isinstance(value, dict):
            return cls(value)
        # All remaining processing is only applied to strings
        if not isinstance(value, str):
            return value
        # Try to interpret `value` as a:
        #   string, number, tuple, list, dict, boolean, or None
        try:
            value = literal_eval(value)
        # The following two excepts allow v to pass through when it represents a
        # string.
        #
        # Longer explanation:
        # The type of v is always a string (before calling literal_eval), but
        # sometimes it *represents* a string and other times a data structure, like
        # a list. In the case that v represents a string, what we got back from the
        # yaml parser is 'foo' *without quotes* (so, not '"foo"'). literal_eval is
        # ok with '"foo"', but will raise a ValueError if given 'foo'. In other
        # cases, like paths (v = 'foo/bar' and not v = '"foo/bar"'), literal_eval
        # will raise a SyntaxError.
        except ValueError:
            pass
        except SyntaxError:
            pass
        return value


load_cfg = (
    CfgNode.load_cfg
)  # keep this function in global scope for backward compatibility


def _valid_type(value, allow_cfg_node=False):
    return (type(value) in _VALID_TYPES) or (
        allow_cfg_node and isinstance(value, CfgNode)
    )


def _merge_a_into_b(a, b, root, key_list):
    """Merge config dictionary a into config dictionary b, clobbering the
    options in b whenever they are also specified in a.
    """
    _assert_with_logging(
        isinstance(a, CfgNode),
        "`a` (cur type {}) must be an instance of {}".format(type(a), CfgNode),
    )
    _assert_with_logging(
        isinstance(b, CfgNode),
        "`b` (cur type {}) must be an instance of {}".format(type(b), CfgNode),
    )

    for k, v_ in a.items():
        full_key = ".".join(key_list + [k])

        v = copy.deepcopy(v_)
        v = b._decode_cfg_value(v)

        if k in b:
            v = _check_and_coerce_cfg_value_type(v, b[k], k, full_key)
            # Recursively merge dicts
            if isinstance(v, CfgNode):
                try:
                    _merge_a_into_b(v, b[k], root, key_list + [k])
                except BaseException:
                    raise
            else:
                b[k] = v
        elif b.is_new_allowed():
            b[k] = v
        else:
            if root.key_is_deprecated(full_key):
                continue
            elif root.key_is_renamed(full_key):
                root.raise_key_rename_error(full_key)
            else:
                raise KeyError("Non-existent config key: {}".format(full_key))


def _check_and_coerce_cfg_value_type(replacement, original, key, full_key):
    """Checks that `replacement`, which is intended to replace `original` is of
    the right type. The type is correct if it matches exactly or is one of a few
    cases in which the type can be easily coerced.
    """
    original_type = type(original)
    replacement_type = type(replacement)

    # The types must match (with some exceptions)
    if replacement_type == original_type:
        return replacement

    # Cast replacement from from_type to to_type if the replacement and original
    # types match from_type and to_type
    def conditional_cast(from_type, to_type):
        if replacement_type == from_type and original_type == to_type:
            return True, to_type(replacement)
        else:
            return False, None

    # Conditionally casts
    # list <-> tuple
    casts = [(tuple, list), (list, tuple)]

    for (from_type, to_type) in casts:
        converted, converted_value = conditional_cast(from_type, to_type)
        if converted:
            return converted_value

    raise ValueError(
        "Type mismatch ({} vs. {}) with values ({} vs. {}) for config "
        "key: {}".format(
            original_type, replacement_type, original, replacement, full_key
        )
    )


def _assert_with_logging(cond, msg):
    if not cond:
        logger.debug(msg)
    assert cond, msg


def _load_module_from_file(name, filename):
    spec = importlib.util.spec_from_file_location(name, filename)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module


In [2]:
cfg = CfgNode(new_allowed=True)
cfg.save_dir = "./"
# common params for NETWORK
cfg.model = CfgNode(new_allowed=True)
cfg.model.arch = CfgNode(new_allowed=True)
cfg.model.arch.fuse = False
cfg.model.arch.ch_l = False
cfg.model.arch.backbone = CfgNode(new_allowed=True)
cfg.model.arch.fpn = CfgNode(new_allowed=True)
cfg.model.arch.head = CfgNode(new_allowed=True)

# DATASET related params
cfg.data = CfgNode(new_allowed=True)
cfg.data.train = CfgNode(new_allowed=True)
cfg.data.train.cache_images = "_"
cfg.data.val = CfgNode(new_allowed=True)
cfg.data.val.cache_images = "_"
cfg.device = CfgNode(new_allowed=True)
cfg.device.precision = 32
cfg.device.batchsize_per_gpu = -1
cfg.device.effective_batchsize = 1
# train
cfg.schedule = CfgNode(new_allowed=True)

# logger
cfg.log = CfgNode()
cfg.log.interval = 50

# testing
cfg.test = CfgNode()
# size of images for each device


def load_config(cfg, args_cfg):
    cfg.defrost()
    cfg.merge_from_file(args_cfg)
    cfg.freeze()

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Dataframe initialization
columns = ["name", "stages_outplanes", "stages_strides", "out_stages", "fpn_in_channels", "fpn_out_stages", "head_input_channel", "head_feat_channels", "head_strides", "aux_head_input_channel", "aux_head_feat_channels", "aux_head_strides", "python", "jit", "trt"]
df = pd.DataFrame(columns=columns)

In [5]:
path_to_dir_yamls = "/home/manos/Thesis/opendr/src/opendr/perception/object_detection_2d/nanodet/algorithm/config/thesis/architecture_search"
name_of_temp_yaml = "temp.yml"
path_to_basic = f"{path_to_dir_yamls}/{name_of_temp_yaml}"
load_config(cfg, path_to_basic)
cfg_base = cfg.clone()
cfg_base.defrost()

In [15]:
features=[3, 3, 5, 5, 9, 9, 17]
yml_outplanes = []
yml_strides = []
yml_names = []
for feat0 in range(features[0]):
    stages_outplanes = []
    stages_strides = []
    if feat0 == 0:
        continue
    stages_outplanes.append(feat0*8)
    stages_strides.append(2)
    for feat1 in range(features[1]):
        for feat2 in range(features[2]):
            for feat3 in range(features[3]):
                for feat4 in range(features[4]):
                    for feat5 in range(features[5]):
                        for feat6 in range(features[6]):
                            yml_names.append(f"{feat0*8}_{feat1*8}_{feat2*8}_{feat3*8}_{feat4*8}_{feat5*8}_{feat6*8}.yml")
                            stages_outplanes = [feat0*8, feat1*8, feat2*8, feat3*8, feat4*8, feat5*8, feat6*8]
                            stages_strides = [2, 1, 2, 1, 2, 1, 2]
                            stages_strides = [stride for feat, stride in zip(stages_outplanes, stages_strides) if feat != 0]
                            stages_outplanes = [feat for feat in stages_outplanes if feat != 0]
                            # yml_outplanes.append([feat0*8, feat1*8, feat2*8, feat3*8, feat4*8, feat5*8, feat6*8])
                            yml_outplanes.append(stages_outplanes)
                            yml_strides.append(stages_strides)

In [22]:
for stages_outplanes, stages_strides, yml_name in zip(yml_outplanes, yml_strides, yml_names):
    if len(stages_outplanes) < 2:
        continue
    else:
        # Find the second output stage of backbone
        if stages_strides[-1] == 1:
            for idx, stride in enumerate(stages_strides):
                if stride == 2:
                    last_2_idx = idx
        else:
            last_2_idx = len(stages_outplanes)-2
        out_stages = [last_2_idx, len(stages_outplanes)-1]
        # name
        name = yml_name
        # configure backbone
        cfg_base.model.arch.backbone.stages_outplanes = stages_outplanes
        cfg_base.model.arch.backbone.stages_strides = stages_strides
        cfg_base.model.arch.backbone.out_stages = out_stages
        cfg_base.model.arch.backbone.stages_kernels = [3 for out in range(len(stages_outplanes))]
        cfg_base.model.arch.backbone.stages_padding = [1 for out in range(len(stages_outplanes))]
        cfg_base.model.arch.backbone.maxpool_kernels = [0 for out in range(len(stages_outplanes))]
        cfg_base.model.arch.backbone.maxpool_strides = [0 for out in range(len(stages_outplanes))]
    
        # configure fpn
        cfg_base.model.arch.fpn.in_channels = [stages_outplanes[out_stages[0]], stages_outplanes[out_stages[1]]]
        cfg_base.model.arch.fpn.out_stages = cfg_base.model.arch.fpn.in_channels[0]
    
        # configure head
        cfg_base.model.arch.head.input_channel = cfg_base.model.arch.fpn.out_stages
        cfg_base.model.arch.head.feat_channels = cfg_base.model.arch.fpn.out_stages
        divisions = np.prod(stages_strides)
        strides = [int(divisions/2), divisions]
        cfg_base.model.arch.head.strides = strides
    
        # configure aux_head
        cfg_base.model.arch.aux_head.input_channel = cfg_base.model.arch.head.input_channel * 2
        cfg_base.model.arch.aux_head.feat_channels = cfg_base.model.arch.head.feat_channels * 2
        cfg_base.model.arch.aux_head.strides = strides
    
        # add to dataframe for dataset
        new_df_row = pd.Series({
            "name": name,
            "stages_outplanes": cfg_base.model.arch.backbone.stages_outplanes,
            "stages_strides": cfg_base.model.arch.backbone.stages_strides,
            "out_stages": cfg_base.model.arch.backbone.out_stages,
            "fpn_in_channels": cfg_base.model.arch.fpn.in_channels,
            "fpn_out_stages": cfg_base.model.arch.fpn.out_stages,
            "head_input_channel": cfg_base.model.arch.head.input_channel,
            "head_feat_channels": cfg_base.model.arch.head.feat_channels,
            "head_strides": cfg_base.model.arch.head.strides,
            "aux_head_input_channel": cfg_base.model.arch.aux_head.input_channel,
            "aux_head_feat_channels": cfg_base.model.arch.aux_head.feat_channels,
            "aux_head_strides": cfg_base.model.arch.aux_head.strides,
            "python": 0,
            "jit": 0,
            "trt": 0
        })
        df = pd.concat([df, new_df_row.to_frame().T], ignore_index=True)
           
        # write down yml file
        with open(f"{path_to_dir_yamls}/saved_ymls/{name}", "w") as f:
            print(cfg_base, file=f)

                            
                            

#save dataframe
df.reset_index(drop=True)
df.reset_index()
df.to_csv(f"{path_to_dir_yamls}/database.csv")

In [24]:
print(stages_outplanes)

[]


In [20]:
print(name)

8_0_0_0_0_0_8.yml


In [21]:
print(cfg_base)

check_point_name: plus_fast
class_names: ['poaceae', 'brassicaceae']
data:
  train:
    cache_images: _
    input_size: [1536, 1312]
    keep_ratio: False
    pipeline:
      brightness: 0.2
      contrast: [0.8, 1.2]
      flip: 0.5
      hard_pos: 0.0
      hard_pos_ratio: 0.0
      jitter_box: 0.1
      normalize: [[98.454, 104.107, 98.173], [34.798, 31.223, 29.665]]
      perspective: 0.0
      rotation: 0
      saturation: [0.8, 1.1]
      scale: [0.9, 1.1]
      shear: 0
      stretch: [[0.9, 1.1], [0.9, 1.1]]
      translate: 0.0
  val:
    cache_images: _
    input_size: [1536, 1312]
    keep_ratio: False
    pipeline:
      normalize: [[98.454, 104.107, 98.173], [34.798, 31.223, 29.665]]
device:
  batchsize_per_gpu: 32
  effective_batchsize: 1
  gpu_ids: [0]
  precision: 32
  workers_per_gpu: 8
evaluator:
  name: CocoDetectionEvaluator
  save_key: mAP
grad_clip: 35
log:
  interval: 5
model:
  arch:
    aux_head:
      activation: LeakyReLU
      feat_channels: 16
      input_c